# Generate samples for unimodal systems with standard MCMC

When many prior samples are used with The Joker, and the sampler returns one sample, or the samples returned are within the same mode of the posterior, the posterior pdf is likely unimodal. In these cases, we can use standard MCMC (e.g., Metropolis-Hastings or an ensemble sampler like `emcee`) to generate posterior samples for us. In this example, we will use `emcee` to "continue" sampling for data that are very constraining, so that The Joker only returns a single sample.

In [ ]:
import astropy.table as at
from astropy.time import Time
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import corner
import pymc3 as pm
import exoplanet as xo

from thejoker import RVData, JokerPrior, TheJoker, JokerSamples
from thejoker.plot import plot_rv_curves

In [ ]:
data_tbl = at.QTable.read('data.ecsv')

rnd = np.random.RandomState(seed=42)
sub_tbl = data_tbl[rnd.choice(len(data_tbl), size=18, replace=False)]
data = RVData.guess_from_table(sub_tbl, t0=data_tbl.meta['t0'])

In [ ]:
_ = data.plot()

In [ ]:
prior = JokerPrior.default(P_min=2*u.day, P_max=1e3*u.day,
                           sigma_K0=30*u.km/u.s,
                           sigma_v=100*u.km/u.s)

In [ ]:
prior_samples = prior.sample(size=1_000_000)
prior_samples

In [ ]:
joker = TheJoker(prior, random_state=rnd)
joker_samples = joker.rejection_sample(data, prior_samples, 
                                       max_posterior_samples=256)
joker_samples

In [ ]:
joker_samples.tbl

In [ ]:
_ = plot_rv_curves(joker_samples, data=data)

In [ ]:
with prior.model:
    mcmc_init = joker.setup_mcmc(data, joker_samples)
    
    trace = pm.sample(tune=1000, draws=1000, 
                      start=mcmc_init,
                      step=xo.get_dense_nuts_step(target_accept=0.95))

In [ ]:
pm.summary(trace, var_names=prior.par_names)

In [ ]:
mcmc_samples = joker.trace_to_samples(trace, remove_constants=False)
mcmc_samples.tbl.remove_column('s')
mcmc_samples.wrap_K()

In [ ]:
import pickle
with open('true-orbit.pkl', 'rb') as f:
    truth = pickle.load(f)

In [ ]:
df = mcmc_samples.tbl.to_pandas()
truths = []
for name in df.columns:
    truths.append(truth[name].value)
_ = corner.corner(df, truths=truths)